In [1]:
import pandas as pd
import json
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
import umap
import hdbscan
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score
import umap
import hdbscan
from itertools import product
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import re
import html
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer


/Users/teatoscanduplantier/Desktop/anaconda3/envs/applied/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-09 17:12:11.315310: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data 

On concatene toute la data que l'on a traité pour l'instant : textes, features et images

In [2]:

photos = pd.read_csv('embeddings_by_business_outside_inside.csv')
texte=pd.read_csv("restaurants_embeddings_1.2.csv")
features=pd.read_csv("features.csv")
photos.rename(columns={'gbusiness_id':'business_id'}, inplace=True)
df=pd.merge(photos, features, on='business_id', how='outer')
df.rename(columns={'embedding':'image_embedding'}, inplace=True)
df=pd.merge(df, texte, on='business_id', how='outer')
df.rename(columns={'embedding':'texte_embedding'}, inplace=True)



In [3]:
print(df.info())
print(df.head())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5852 entries, 0 to 5851
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   business_id            5852 non-null   object 
 1   image_embedding        2146 non-null   object 
 2   n_outside              2146 non-null   float64
 3   n_inside               2146 non-null   float64
 4   review_count           5852 non-null   int64  
 5   log_review_count       5852 non-null   float64
 6   stars                  5852 non-null   float64
 7   rating_std             5852 non-null   float64
 8   positive_ratio         5852 non-null   float64
 9   negative_ratio         5852 non-null   float64
 10  photo_count            3299 non-null   float64
 11  log_photo_count        3299 non-null   float64
 12  business_age_years     5852 non-null   float64
 13  review_growth_rate     5852 non-null   float64
 14  expert_reviewer_ratio  5852 non-null   float64
 15  elit

In [4]:
data = []
with open("philly_restaurant_reviews.json", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
texte_original = pd.DataFrame(data)

# nombre d'avis par business
textes_40 = (
    texte_original
    .sort_values("date", ascending=False)   # du plus récent au plus ancien
    .groupby("business_id")
    .head(40)                                # garder les 40 plus récents
    .groupby("business_id")["text"]
    .apply(lambda x: " ".join(x))            # concatener les textes
    .reset_index()
)

In [5]:
df=pd.merge(df, textes_40, on='business_id', how='outer')

On supprime les restaurants fermés (inutile pour nous) et on garde que ceux qui ont plus de 5 reviews

In [6]:
df = df[df['is_open'] != 0]
# On ne garde que ceux qui ont 5 reviews ou plus
df = df[df['n_reviews'] >= 5]
df = df.reset_index(drop=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3525 entries, 0 to 3524
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   business_id            3525 non-null   object 
 1   image_embedding        1426 non-null   object 
 2   n_outside              1426 non-null   float64
 3   n_inside               1426 non-null   float64
 4   review_count           3525 non-null   int64  
 5   log_review_count       3525 non-null   float64
 6   stars                  3525 non-null   float64
 7   rating_std             3525 non-null   float64
 8   positive_ratio         3525 non-null   float64
 9   negative_ratio         3525 non-null   float64
 10  photo_count            2186 non-null   float64
 11  log_photo_count        2186 non-null   float64
 12  business_age_years     3525 non-null   float64
 13  review_growth_rate     3525 non-null   float64
 14  expert_reviewer_ratio  3525 non-null   float64
 15  elit

Code pour converdit les embedding en le nombre de dimension de base

In [8]:
# Étape 1: Convertir les strings en vecteurs numériques
def convert_embeddings(embeddings_strings):
    """
    Convertit un array de strings d'embeddings en array numérique XD
    Les embeddings 'nan' sont gardés comme vecteurs de NaN
    """
    embeddings_list = []
    embedding_dim = None
    
    # Premier passage pour déterminer la dimension
    for emb_str in embeddings_strings:
        if isinstance(emb_str, np.ndarray):
            emb_str = emb_str[0]
        
        if emb_str != 'nan' and emb_str != 'None':
            try:
                emb_str_clean = emb_str.strip('[]')
                emb_array = np.fromstring(emb_str_clean, sep=' ')
                embedding_dim = len(emb_array)
                break
            except:
                continue
    
    if embedding_dim is None:
        raise ValueError("Impossible de déterminer la dimension des embeddings")
    
    print(f" Dimension des embeddings: {embedding_dim}")
    
    # Deuxième passage pour convertir tous les embeddings
    for i, emb_str in enumerate(embeddings_strings):
        # Extraire la string (elle est dans un array de shape (1,))
        if isinstance(emb_str, np.ndarray):
            emb_str = emb_str[0]
        
        # Si c'est nan, créer un vecteur de NaN
        if emb_str == 'nan' or emb_str == 'None':
            nan_vector = np.full(embedding_dim, np.nan)
            embeddings_list.append(nan_vector)
            continue
        
        try:
            # Nettoyer la string et convertir en array
            emb_str_clean = emb_str.strip('[]')
            emb_array = np.fromstring(emb_str_clean, sep=' ')
            embeddings_list.append(emb_array)
        except Exception as e:
            print(f"Erreur pour l'embedding {i}: {e} - remplacé par NaN")
            nan_vector = np.full(embedding_dim, np.nan)
            embeddings_list.append(nan_vector)
    
    # Convertir en array NumPy 2D
    embeddings_2d = np.array(embeddings_list)
    
    # Compter les NaN
    n_nan = np.isnan(embeddings_2d).any(axis=1).sum()
    print(f"✓ {n_nan} embeddings avec NaN gardés")
    
    return embeddings_2d


In [9]:
image_embedding = convert_embeddings(df['image_embedding'].tolist())
texte_embedding=convert_embeddings(df['texte_embedding'].tolist())
features = df[['log_review_count', 'stars', 'rating_std', 'positive_ratio', 
               'negative_ratio',
               'business_age_years', 'review_growth_rate', 'expert_reviewer_ratio', 
               'elite_reviewer_ratio', 'high_influence_ratio', 'mean_influence_score']]

 Dimension des embeddings: 512
Erreur pour l'embedding 2: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 3: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 5: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 8: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 9: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 11: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 13: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 14: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 15: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 16: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'embedding 20: 'float' object has no attribute 'strip' - remplacé par NaN
Erreur pour l'emb

# Clustering en incluant les photos en gardant les paramètres du premier grid search 

Le grid search etant un très long processus, on émet l'hypothèse que les paramètres trouvés lors du premier grid search sont également optimisés pour le clustering en ajoutant les images. Aussi, cela nous permet d'avoir une comparaison équitable.

In [10]:
# Option : Remplacer NaN par zéros (restaurants sans photos = vecteur neutre)
image_embedding_filled = image_embedding.copy()
has_image = ~np.isnan(image_embedding_filled).any(axis=1)
image_embedding_filled[np.isnan(image_embedding_filled)] = 0


# Réduction de dimensionnalité pour chaque modalité
# Texte 
pca_text = PCA(n_components=50)
text_reduced = pca_text.fit_transform(texte_embedding)

# Images
pca_image = PCA(n_components=50)
image_reduced = pca_image.fit_transform(image_embedding_filled)


# Normalisation
text_scaled = StandardScaler().fit_transform(text_reduced)
image_scaled = StandardScaler().fit_transform(image_reduced)
features_scaled = RobustScaler().fit_transform(features)

# Poids : alpha (texte) + beta (images) + gamma (features) = 1
alpha = 0.3   
beta = 0.2    
gamma = 0.5 

# Fusion pondérée
combined_multimodal = np.concatenate([
    alpha * text_scaled,
    beta * image_scaled,
    gamma * features_scaled
], axis=1)

# UMAP + HDBSCAN
print("\n UMAP projection")
reducer = umap.UMAP(
    n_components=30,
    n_neighbors=10,
    min_dist=0.0,
    metric='euclidean',
    random_state=42,
    verbose=False
)
embedded_multimodal = reducer.fit_transform(combined_multimodal)

print(" Clustering HDBSCAN")
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=60,
    min_samples=30,
    metric="euclidean"
)
labels_multimodal = clusterer.fit_predict(embedded_multimodal)

#sauvegarde
umap_cols = [f'umap_multimodal_{i}' for i in range(embedded_multimodal.shape[1])]
df_umap_new = pd.DataFrame(embedded_multimodal, columns=umap_cols, index=df.index)

df_final_multimodal = pd.concat([df, df_umap_new], axis=1)
df_old = pd.read_csv('resultats_clustering_complet.csv')
labels_old = df_old['cluster_label'].values
df_final_multimodal['cluster_label_old'] = labels_old
df_final_multimodal['cluster_label_multimodal'] = labels_multimodal

df_final_multimodal.to_csv('resultats_inside_outside.csv', index=False)

print(f"\n Sauvegardé dans: resultats_inside_outside.csv")



 UMAP projection
 Clustering HDBSCAN

 Sauvegardé dans: resultats_inside_outside.csv


## Analyse des résultats et comparaison

In [11]:
n_clusters_new = len(set(labels_multimodal)) - (1 if -1 in labels_multimodal else 0)
n_outliers_new = (labels_multimodal == -1).sum()

print(f"\n{'='*80}")
print("📊 COMPARAISON CLUSTERING OUTSIDE/INSIDE")
print(f"{'='*80}")

# Charger ancien clustering (toutes images)
df_old = pd.read_csv('resultats_clustering_multimodal.csv')

# ⚠️ VÉRIFICATION CRITIQUE : S'assurer que les business_id correspondent
print(f"\n🔍 VÉRIFICATIONS DE COHÉRENCE:")
print(f"  Restaurants actuels (outside/inside): {len(df)}")
print(f"  Restaurants ancien clustering (toutes images): {len(df_old)}")

# Vérifier si les tailles sont identiques
if len(df_old) != len(df):
    print(f"\n⚠️  ATTENTION: Tailles différentes!")
    print(f"     → Merger sur business_id pour aligner les données")
    
    # Créer un merge pour aligner correctement
    df_aligned = df[['business_id']].copy()
    df_aligned['cluster_label_new'] = labels_multimodal
    
    df_comparison_full = pd.merge(
        df_old[['business_id', 'cluster_label_multimodal']],
        df_aligned,
        on='business_id',
        how='inner'
    )
    
    labels_old = df_comparison_full['cluster_label_multimodal'].values
    labels_new_aligned = df_comparison_full['cluster_label_new'].values
    
    print(f"     → {len(df_comparison_full)} restaurants communs trouvés")
    
else:
    # Vérifier que les business_id sont dans le même ordre
    if (df_old['business_id'].values == df['business_id'].values).all():
        labels_old = df_old['cluster_label_multimodal'].values
        labels_new_aligned = labels_multimodal
    else:
        print(f"business_id pas dans le même ordre!")

        
        # Réaligner via merge
        df_aligned = df[['business_id']].copy()
        df_aligned['cluster_label_new'] = labels_multimodal
        
        df_comparison_full = pd.merge(
            df_old[['business_id', 'cluster_label_multimodal']],
            df_aligned,
            on='business_id',
            how='inner'
        )
        
        labels_old = df_comparison_full['cluster_label_multimodal'].values
        labels_new_aligned = df_comparison_full['cluster_label_new'].values
        

# Calculer les statistiques
n_clusters_old = len(set(labels_old)) - (1 if -1 in labels_old else 0)
n_outliers_old = (labels_old == -1).sum()

# Silhouette nouveau
mask_new = labels_multimodal != -1
if mask_new.sum() > 0:
    sil_new = silhouette_score(embedded_multimodal[mask_new], labels_multimodal[mask_new])
    db_new = davies_bouldin_score(embedded_multimodal[mask_new], labels_multimodal[mask_new])
    print(f"  Silhouette: {sil_new:.3f}")
    print(f"  Davies-Bouldin: {db_new:.3f}")




📊 COMPARAISON CLUSTERING OUTSIDE/INSIDE

🔍 VÉRIFICATIONS DE COHÉRENCE:
  Restaurants actuels (outside/inside): 3525
  Restaurants ancien clustering (toutes images): 3525
  Silhouette: 0.614
  Davies-Bouldin: 0.533


In [12]:
df_multi = pd.read_csv('resultats_clustering_outside_inside.csv')
df_multi.columns

Index(['business_id', 'image_embedding', 'n_outside', 'n_inside',
       'review_count', 'log_review_count', 'stars', 'rating_std',
       'positive_ratio', 'negative_ratio', 'photo_count', 'log_photo_count',
       'business_age_years', 'review_growth_rate', 'expert_reviewer_ratio',
       'elite_reviewer_ratio', 'high_influence_ratio', 'mean_influence_score',
       'cluster_ambiance', 'is_open', 'texte_embedding', 'n_reviews', 'text',
       'umap_outside_inside_0', 'umap_outside_inside_1',
       'umap_outside_inside_2', 'umap_outside_inside_3',
       'umap_outside_inside_4', 'umap_outside_inside_5',
       'umap_outside_inside_6', 'umap_outside_inside_7',
       'umap_outside_inside_8', 'umap_outside_inside_9',
       'umap_outside_inside_10', 'umap_outside_inside_11',
       'umap_outside_inside_12', 'umap_outside_inside_13',
       'umap_outside_inside_14', 'umap_outside_inside_15',
       'umap_outside_inside_16', 'umap_outside_inside_17',
       'umap_outside_inside_18', 'uma

Cela aurait été intéressant de continuer sur cette piste mais par manque de temps nous ne pouvons pas